In [1]:
import lmfit
import numpy as np
from numpy import exp, linspace, random
from scipy.optimize import curve_fit, leastsq
from numpy import ma

### Model

In [2]:
def gaussian(params, x, y):
    amp, cen, wid = params[0], params[1], params[2]
    residual = y - amp * exp(-(x-cen)**2 / wid)
    return residual

### Residual

In [3]:
def gaussian_g(params, x, data):
    height, center, sigma, c = params[0], params[1], params[2], params[3]
    
    model = height * np.exp(-((1.0 * x - center) ** 2) / max(1.e-15, (2 * sigma ** 2))) + c
    return data-model

### Data

In [32]:
x1= linspace(-10, 10, 101)
y1 = 2.33 * exp(-(x1-0.21)**2 / 1.51)+ random.normal(0, 0.2, x1.size)

In [4]:
xx = np.array([6548.06489762, 6549.31489762, 6550.56489762, 6551.81489762,
       6553.06489762, 6554.31489762, 6555.56489762, 6556.81489762,
       6558.06489762, 6559.31489762, 6560.56489762, 6561.81489762,
       6563.06489762, 6564.31489762, 6565.56489762, 6566.81489762,
       6568.06489762, 6569.31489762, 6570.56489762, 6571.81489762,
       6573.06489762, 6574.31489762, 6575.56489762, 6576.81489762,
       6578.06489762])

In [90]:
# data = ma.MaskedArray(data=[  7.48501778,   7.09317732,   7.03931999,   7.03201437,
#          7.12849331,   7.44794178,   7.16412687,   7.71622229,
#          9.18051338,  21.93631554,  66.16438293, 140.59335327,
#        179.75015259, 134.93383789,  61.15007019,  20.12806702,
#          8.98960495,   7.18552256,   7.04587221,   7.17096138,
#          7.05737543,   7.03616285,   7.3354826 ,   7.11767721,
#          7.28576183],
#              mask=[False,  True,  True, False, False, False, False, False,
#                    False, False, False,  True,  True, False, False, False,
#                    False, False, False,  True, False, False, False, False,
#                    False],
#        fill_value=1e+20)

In [92]:
# non-masked array
data = np.array([  7.48501778,   7.09317732,   7.03931999,   7.03201437,
         7.12849331,   7.44794178,   7.16412687,   7.71622229,
         9.18051338,  21.93631554,  66.16438293, 140.59335327,
       179.75015259, 134.93383789,  61.15007019,  20.12806702,
         8.98960495,   7.18552256,   7.04587221,   7.17096138,
         7.05737543,   7.03616285,   7.3354826 ,   7.11767721,
         7.28576183])

### Initial guess

In [6]:
gau_init = [127.900000 , 6563.06000, 1.25000000 , 7.24000000 ]

### transform to internal parameter space

In [9]:
# paramers height and sigma have a minminum value of zero
# height_internal = np.sqrt((height+1)**2 -1)
# sigma_internal = np.sqrt((sigma+1)**2 -1)
# height = np.sqrt(height_internal**2 + 1) - 1
# sigma = np.sqrt(sigma_internal**2 + 1) -1

In [94]:
def gaussian_g_internal(params, x, data):
    height_internal, center, sigma_internal, c = params[0], params[1], params[2], params[3]
    
    model = (np.sqrt(height_internal**2 + 1) - 1) * np.exp(-((1.0 * x - center) ** 2) / max(1.e-15, (2 * (np.sqrt(sigma_internal**2 + 1) -1) ** 2))) + c
    return data-model

In [95]:
gau_init_internal = [ np.sqrt((127.900000+1)**2 -1), 6563.06000,  np.sqrt((1.25000000+1)**2 -1), 7.24000000 ]

In [96]:
gau_init_internal

[128.89612096568308, 6563.06, 2.0155644370746373, 7.24]

### leastsq fitting result

#### fit for internal parameters

In [97]:
gau_internal_res = leastsq(gaussian_g_internal, gau_init_internal, (xx, data), full_output=1)

In [98]:
gau_internal_res[0]

array([1.73715919e+02, 6.56301600e+03, 2.47696679e+00, 7.17351852e+00])

#### convert to external (bounded) parameters

In [103]:
gau_res_0 = [np.sqrt(1.73715919e+02**2 + 1) - 1, 6.56301600e+03, np.sqrt(2.47696679e+00**2 + 1) -1, 7.17351852e+00]

In [104]:
gau_res_0

[172.71879723856762, 6563.016, 1.6712103022343459, 7.17351852]

#### covariance matrix transformation

In [101]:
cov_internal = gau_internal_res[1]

In [102]:
cov_internal.shape

(4, 4)

##### scale_gradient

In [105]:
height_scale = 172.71879723856762/np.sqrt(172.71879723856762**2 + 1)
center_scale = 1.
sigma_scale = 1.6712103022343459/np.sqrt(1.6712103022343459**2 + 1)
c_scale = 1.

In [106]:
g = [height_scale, center_scale, sigma_scale, c_scale]

In [107]:
grad2d = np.atleast_2d(g)

##### calcuate external covariance matrix

In [108]:
grad = np.dot(grad2d.T, grad2d)

In [109]:
cov_ext = cov_internal * grad

In [110]:
cov_ext

array([[ 6.60925132e-01,  2.33257393e-07, -3.27798474e-03,
        -3.95242958e-02],
       [ 2.33257393e-07,  7.90177406e-05, -2.01819422e-09,
        -1.03074354e-09],
       [-3.27798474e-03, -2.01819422e-09,  7.66292623e-05,
        -7.07785822e-04],
       [-3.95242958e-02, -1.03074354e-09, -7.07785822e-04,
         5.58945054e-02]])

#### calculate chi2

In [111]:
residual = gaussian_g([172.71879723856762, 6563.016, 1.6712103022343459, 7.17351852], xx, data)

In [112]:
residual

array([ 0.31149926, -0.0803412 , -0.13419853, -0.14150418, -0.04502866,
        0.27419884, -0.01772511,  0.36584615, -0.13815674, -0.10778912,
        0.0747532 ,  0.01365505, -0.06824863,  0.06723987, -0.00146206,
       -0.0861942 ,  0.01545991, -0.13009163, -0.13405507, -0.00272234,
       -0.11614552, -0.13735569,  0.16196408, -0.05584131,  0.11224331])

In [113]:
chi2 = (residual**2).sum()

In [114]:
chi2

0.5154268353809002

In [116]:
ndata = len(residual)
nfree = ndata - 4

In [117]:
ndata

25

In [118]:
reduced_chi2 = chi2/nfree

In [119]:
reduced_chi2

0.024544135018138104

#### calculate stderr

##### scale covariance matrix

In [120]:
cov_ext_scale = cov_ext * reduced_chi2

In [121]:
cov_ext_scale

array([[ 1.62218357e-02,  5.72510095e-09, -8.04553000e-05,
        -9.70089653e-04],
       [ 5.72510095e-09,  1.93942209e-06, -4.95348315e-11,
        -2.52987085e-11],
       [-8.04553000e-05, -4.95348315e-11,  1.88079896e-06,
        -1.73719908e-05],
       [-9.70089653e-04, -2.52987085e-11, -1.73719908e-05,
         1.37188229e-03]])

##### calculate stderr

In [122]:
height_err = np.sqrt(cov_ext_scale[0,0])

In [123]:
height_err  # output from lmfit: 0.12736497

0.1273649703649372

In [124]:
center_err = np.sqrt(cov_ext_scale[1,1]) # output from lmfit: 0.00139263 

In [125]:
center_err

0.0013926313563530622

In [126]:
sigma_err = np.sqrt(cov_ext_scale[2,2]) # output from lmfit: 0.00148197 

In [127]:
sigma_err

0.001371422240276804

In [128]:
c_err = np.sqrt(cov_ext_scale[3,3]) # output from lmfit: 0.03703892 

In [129]:
c_err

0.03703892933752733

### uncategorized

In [7]:
gau_res = leastsq(gaussian_g, gau_init, (xx, data), full_output=1)

In [8]:
gau_res[0]

array([1.72999600e+02, 6.56301520e+03, 1.66946370e+00, 7.19380093e+00])

In [162]:
errorbars = (gau_res[1] is not None)

In [164]:
grad = np.array([[0.99996648, 0.99998324, 0.85809549, 0.99998324],
       [0.99998324, 1.        , 0.85810987, 1.        ],
       [0.85809549, 0.85810987, 0.73635254, 0.85810987],
       [0.99998324, 1.        , 0.85810987, 1.        ]])

In [165]:
cov_ext = gau_res[1]*grad

In [166]:
cov_ext

array([[ 4.85238718e+00, -1.55110596e-02, -2.02136018e-02,
         1.96286738e-02],
       [-1.55110596e-02,  1.60043134e-04,  5.30489461e-05,
        -1.57657757e-04],
       [-2.02136018e-02,  5.30489461e-05,  1.43445119e-04,
        -1.10466444e-03],
       [ 1.96286738e-02, -1.57657757e-04, -1.10466444e-03,
         6.85435462e-02]])

In [170]:
np.sqrt(cov_ext[3,3])

0.2618082240206693

In [171]:
init_vals = [1, 0, 1]  # for [amp, cen, wid]
res = leastsq(gaussian,init_vals, (x1,y1), full_output=1)

In [55]:
res[0]

array([2.56981074, 0.17804159, 1.35999017])

In [146]:
res[0]

array([2.56981074, 0.17804159, 1.35999017])

In [148]:
res[1].shape

(3, 3)

In [150]:
res[1]

array([[ 2.05253236e-01,  2.53932497e-09, -1.44834482e-01],
       [ 2.53932497e-09,  2.81801036e-02, -1.42530474e-09],
       [-1.44834482e-01, -1.42530474e-09,  3.06602144e-01]])

In [109]:
# test with masked array

_ma = np.random.randint(2, size= x1.size, dtype=np.bool_)

In [110]:
y2 = ma.MaskedArray(y1, mask=_ma)

In [118]:
%%timeit
res2 = leastsq(gaussian, init_vals, (x1,y2), full_output=1)

1.54 ms ± 74.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [113]:
res2 = leastsq(gaussian, init_vals, (x1,y2), full_output=1)

In [114]:
res2[0]

array([2.45893809, 0.20409826, 1.44003284])

In [81]:
res2[2]['fvec']

101

In [115]:
y3 = y2[y2.mask == False].copy()
x3 = x1[y2.mask == False].copy()
res3 = leastsq(gaussian, init_vals, (x3,y3), full_output=1)

In [119]:
%%timeit
y3 = y2[y2.mask == False].copy()
x3 = x1[y2.mask == False].copy()
res3 = leastsq(gaussian, init_vals, (x3,y3), full_output=1)

1.65 ms ± 25.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [120]:
res3

(array([2.45892325, 0.20409685, 1.44005681]),
 array([[ 0.65542453,  0.03029124, -0.43253447],
        [ 0.03029124,  0.06628258, -0.03883975],
        [-0.43253447, -0.03883975,  0.69802139]]),
 {'fvec': array([ 0.03477273, -0.12872887,  0.07137988,  0.07652104,  0.02078202,
         -0.34226198, -0.18632205, -0.49955337,  0.05002334,  0.11161321,
         -0.16067391,  0.20749805, -0.16828137,  0.11033501,  0.07547328,
          0.22427452,  0.21117658, -0.26676398,  0.00084459, -0.19886104,
          0.2644002 , -0.14889203,  0.03400941, -0.09867757,  0.04668435,
          0.10326536, -0.3245153 ,  0.25123061,  0.11343694,  0.06027532,
         -0.1323442 , -0.10097297, -0.02547032,  0.38950465, -0.08525189,
          0.03983859, -0.10156034, -0.18731067,  0.04922391, -0.04462   ,
         -0.17696021,  0.18533388, -0.02355286,  0.01408838, -0.4398535 ,
          0.07842244,  0.07664051,  0.21585108,  0.02755452, -0.1063773 ,
          0.31768274,  0.12692786]),
  'nfev': 25,
  'fja

In [91]:
y2

masked_array(data=[--, 0.03477273280950199, -0.1287288747298011, --,
                   0.07652104108266816, --, --, 0.20076604963149008,
                   0.020782016195516027, --, -0.11210832292164126,
                   -0.3422619760649085, --, --, 0.13234255505703996,
                   0.2401563719276388, --, -0.09029294666185246,
                   0.11161320686430481, 0.15102122721917077,
                   -0.16067390775758464, 0.14023479276815742,
                   0.26819891931636985, 0.20749805543800504,
                   -0.13153144084138627, --, --, -0.20246972713988165, --,
                   --, 0.07548477591632655, --, --, -0.06989248899210396,
                   0.2119637291890957, -0.2647931959034779,
                   0.005512187937790004, --, -0.11866444217425787, --,
                   0.34866557574940993, --, 0.10765557346101168,
                   0.26624659954730856, --, --, 1.1222428336411066,
                   2.0050230224886207, 1.9551657571368004, --,
 

In [122]:
from lmfit import Minimizer, Parameters, report_fit

In [124]:
report_fit

<function lmfit.printfuncs.report_fit(params, **kws)>

In [125]:
x = np.linspace(0, 15, 301)
np.random.seed(2021)
data = (5.0 * np.sin(2.0*x - 0.1) * np.exp(-x*x*0.025) +
        np.random.normal(size=x.size, scale=0.2))

In [126]:
def fcn2min(params, x, data):
    """Model a decaying sine wave and subtract data."""
    amp = params['amp']
    shift = params['shift']
    omega = params['omega']
    decay = params['decay']
    model = amp * np.sin(x*omega + shift) * np.exp(-x*x*decay)
    return model - data

In [127]:
params = Parameters()
params.add('amp', value=10, min=0)
params.add('decay', value=0.1)
params.add('shift', value=0.0, min=-np.pi/2., max=np.pi/2.)
params.add('omega', value=3.0)

In [128]:
minner = Minimizer(fcn2min, params, fcn_args=(x, data))

In [129]:
minner

In [131]:
result = minner.minimize()


In [132]:
result.method

'leastsq'

In [134]:
final = data + result.residual

In [136]:
report_fit(result)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 59
    # data points      = 301
    # variables        = 4
    chi-square         = 12.1867036
    reduced chi-square = 0.04103267
    Akaike info crit   = -957.236198
    Bayesian info crit = -942.407756
[[Variables]]
    amp:    5.03087926 +/- 0.04005805 (0.80%) (init = 10)
    decay:  0.02495454 +/- 4.5395e-04 (1.82%) (init = 0.1)
    shift: -0.10264935 +/- 0.01022298 (9.96%) (init = 0)
    omega:  2.00026304 +/- 0.00326184 (0.16%) (init = 3)
[[Correlations]] (unreported correlations are < 0.100)
    C(shift, omega) = -0.785
    C(amp, decay)   = 0.584
    C(amp, shift)   = -0.118
